In [16]:
import os
import pandas as pd
import numpy as np

# Cleaning the Data - Steps

## Step 1: Read the data

Some things are going mighty wrong with this datafiles.

+ They are using whitespace to delimit new entries
+ And the number of spaces is inconsistent...
+ And their value for `NA` is `-9999`

All in all that's not the worst I've seen as far as unorthodox data-management goes...

If that was not enough, it is also possible for the first column to be missing - which is not delimited and thus reading the file becomes a chore.

### Step 1.1: Replace the codes for missing data

We try this step before reading the data into `Python`, in order to avoid parsing issues. We can achieve this by an old staple of the `UNIX` community: the *Stream-EDitor* `sed`

In [17]:
!sed -i 's/-9999/ NA NA /g' ../data/example.dly

In [18]:
!sed -i 's/ 0T/ 0/g' ../data/example.dly

#### Explanation:

```sed -i 's/original/new/g' file.txt```

`sed` = Stream EDitor
`-i` = in-place (i.e. save back to the original file)

The command string:

`s` = the substitute command
`original` = a regular expression describing the word to replace (or just the word itself)
`new` = the text to replace it with
`g` = global (i.e. replace all and not just the first occurrence)
`file.txt` = the file name

Here we need to replace their code for missing values (`-9999`) with two `NA`s. One for the missing measurement and one for the missing quality flag.

### Step 1.2: Read the data, avoiding the quality-flags

Every second column in the raw data contains a measurement that indicates the quality of the measurement. While it could have interesting implications, I think it best to simply ignore these values and hope that all the variance in the obtained measures is simply (and strictly) random.

In [19]:
data_dir = os.path.join("..", "data", "ncdc", "ghcnd_all")
example_path = os.path.join("..", "data", "example.dly")
df = pd.read_csv(example_path, delim_whitespace=True, header=None, engine="python", index_col=[0])

Thus it is best to simply take every second column and discard the rest.

In [20]:
df = df.iloc[:, ::2]
assert df.shape[1] == 31
df.columns = range(1, 32)

In [21]:
df.head(10)

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
0,,,,,,,,,,,,,,,,,,,,,
ACW00011604194901TMAX,289,289.0,283.0,283.0,289.0,289.0,278.0,267.0,272.0,278,...,272.0,267.0,267.0,267.0,278.0,272.0,272.0,272.0,272.0,272.0
ACW00011604194901TMIN,217,228.0,222.0,233.0,222.0,222.0,228.0,217.0,222.0,183,...,217.0,211.0,211.0,200.0,222.0,217.0,211.0,222.0,206.0,217.0
ACW00011604194901PRCP,0,30.0,0.0,0.0,25.0,41.0,0.0,0.0,0.0,0,...,0.0,53.0,0.0,10.0,15.0,41.0,0.0,86.0,28.0,15.0
ACW00011604194901SNOW,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACW00011604194901SNWD,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACW00011604194901WT16,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0,NaN,NaN,...,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ACW00011604194902TMAX,267,278.0,272.0,267.0,278.0,278.0,278.0,267.0,278.0,267,...,272.0,272.0,267.0,267.0,267.0,267.0,272.0,NaN,NaN,NaN
ACW00011604194902TMIN,211,211.0,222.0,206.0,211.0,233.0,211.0,233.0,206.0,211,...,200.0,211.0,211.0,200.0,211.0,217.0,222.0,NaN,NaN,NaN
ACW00011604194902PRCP,0,15.0,0.0,64.0,53.0,5.0,3.0,18.0,41.0,18,...,89.0,0.0,0.0,18.0,0.0,0.0,5.0,NaN,NaN,NaN


## Step 2 Clean the data

This might seem easier than it is - as usual :)

A couple of things are amiss here.

1. The index contains information which should be stored in a column.
1. The rows should really be columns.

#### Step 2.1: Fix the index

From the documentation we know that the first is comprised by multiple building blocks which are identified by their position in the string:

|Variable  | Columns |  Type      |
|----------|---------|------------|
|ID        |    1-11 |  Character |
|YEAR      |   12-15 |  Integer   |
|MONTH     |   16-17 |  Integer   |
|ELEMENT   |   18-21 |  Character |

Thus we can extract them easily by a `regex`.

In [22]:
id_vars = df.index.str.extract("^([A-Z0-9]{11})(\d{4})(\d{2})(.*)$", expand=True).set_index(df.index)
id_vars.columns = ["station", "year", "month", "measure"]
id_vars.head()

,station,year,month,measure
0,,,,
ACW00011604194901TMAX,ACW00011604,1949,01,TMAX
ACW00011604194901TMIN,ACW00011604,1949,01,TMIN
ACW00011604194901PRCP,ACW00011604,1949,01,PRCP
ACW00011604194901SNOW,ACW00011604,1949,01,SNOW
ACW00011604194901SNWD,ACW00011604,1949,01,SNWD


Now merge them back into the original dataframe, drop the old index and reindex the whole thing.

In [23]:
df = pd.concat([df, id_vars], axis=1)
df.head()

,1,2,3,4,5,6,7,8,9,10,...,26,27,28,29,30,31,station,year,month,measure
0,,,,,,,,,,,,,,,,,,,,,
ACW00011604194901TMAX,289,289.0,283.0,283.0,289.0,289.0,278.0,267.0,272.0,278,...,278.0,272.0,272.0,272.0,272.0,272.0,ACW00011604,1949,01,TMAX
ACW00011604194901TMIN,217,228.0,222.0,233.0,222.0,222.0,228.0,217.0,222.0,183,...,222.0,217.0,211.0,222.0,206.0,217.0,ACW00011604,1949,01,TMIN
ACW00011604194901PRCP,0,30.0,0.0,0.0,25.0,41.0,0.0,0.0,0.0,0,...,15.0,41.0,0.0,86.0,28.0,15.0,ACW00011604,1949,01,PRCP
ACW00011604194901SNOW,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,ACW00011604,1949,01,SNOW
ACW00011604194901SNWD,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,ACW00011604,1949,01,SNWD


### Step 2.2: Fix the whole rows-are-columns-situation

In [24]:
long = pd.melt(df, id_vars=["station", "year", "month", "measure"], var_name="day")
long.head()

,station,year,month,measure,day,value
0,ACW00011604,1949,01,TMAX,1,289
1,ACW00011604,1949,01,TMIN,1,217
2,ACW00011604,1949,01,PRCP,1,0
3,ACW00011604,1949,01,SNOW,1,0
4,ACW00011604,1949,01,SNWD,1,0


In [25]:
long["id"] = long.year.astype(str) + "-" + long.month.astype(str) + "-" + long.day.astype(str).str.pad(2, fillchar="0")

In [26]:
out = long.pivot(index="id", columns="measure", values="value")

In [27]:
valid_dates = pd.date_range("1949-01-01", "1949-08-31").astype(str)

In [28]:
out = out.loc[valid_dates]

In [29]:
out.index.name = "date"

In [30]:
out

measure,PGTM,PRCP,SNOW,SNWD,TMAX,TMIN,WDFG,WSFG,WT03,WT08,WT16
date,,,,,,,,,,,
1949-01-01,NaN,0,0,0,289,217,NaN,NaN,NaN,NaN,NaN
1949-01-02,NaN,30,0,0,289,228,NaN,NaN,NaN,NaN,1
1949-01-03,NaN,0,0,0,283,222,NaN,NaN,NaN,NaN,NaN
1949-01-04,NaN,0,0,0,283,233,NaN,NaN,NaN,NaN,1
1949-01-05,NaN,25,0,0,289,222,NaN,NaN,NaN,NaN,1
1949-01-06,NaN,41,0,0,289,222,NaN,NaN,NaN,NaN,1
1949-01-07,NaN,0,0,0,278,228,NaN,NaN,NaN,NaN,NaN
1949-01-08,NaN,0,0,0,267,217,NaN,NaN,NaN,NaN,1
1949-01-09,NaN,0,0,0,272,222,NaN,NaN,NaN,NaN,NaN


In [31]:
out.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243 entries, 1949-01-01 to 1949-08-31
Data columns (total 11 columns):
PGTM    4 non-null object
PRCP    225 non-null object
SNOW    226 non-null object
SNWD    226 non-null object
TMAX    212 non-null object
TMIN    212 non-null object
WDFG    4 non-null object
WSFG    4 non-null object
WT03    4 non-null object
WT08    1 non-null object
WT16    113 non-null object
dtypes: object(11)
memory usage: 22.8+ KB


In [32]:
for c in out.columns:
    try:
        out[c].astype(float)
    except:
        print(c)
        print(out[c].value_counts())

WDFG
68W     3
113W    1
Name: WDFG, dtype: int64


In [63]:
d_ranges = long.loc[:, ["year", "month", "day"]].sort_values(["year", "month", "day"]).iloc[[0, -1]]
d_ranges

,year,month,day
0,1949,01,1
1518,1949,08,31


In [64]:
start, end = pd.to_datetime(d_ranges)

In [35]:
pd.date_range(start, end)

DatetimeIndex(['1949-01-01', '1949-01-02', '1949-01-03', '1949-01-04',
               '1949-01-05', '1949-01-06', '1949-01-07', '1949-01-08',
               '1949-01-09', '1949-01-10',
               ...
               '1949-08-22', '1949-08-23', '1949-08-24', '1949-08-25',
               '1949-08-26', '1949-08-27', '1949-08-28', '1949-08-29',
               '1949-08-30', '1949-08-31'],
              dtype='datetime64[ns]', length=243, freq='D')

In [39]:
min_year, max_year = long.year.min(), long.year.max()

In [37]:
long.groupby("year").agg({"month": ["min", "max"]})

month    
       min max
year          
1949    01  08

In [40]:
test = pd.DataFrame({"year": [1991, 1995] * 10, "month": np.random.randint(1, 13, size=20), "day": np.random.randint(1, 32, size=20)})

In [41]:
test

,year,month,day
0,1991,5,28
1,1995,2,18
2,1991,12,16
3,1995,6,9
4,1991,12,5
5,1995,12,16
6,1991,7,2
7,1995,9,21
8,1991,9,1
9,1995,7,3


In [44]:
start_year = test.year.min()
print(start_year)

1991


In [46]:
start_month = test.loc[test.year == start_year].month.min()
print(start_month)

1


In [48]:
test.loc[(test.year == start_year) & (test.month == start_month)].day.min()

19

In [50]:
test.sort_values(["year", "month", "day"])

,year,month,day
16,1991,1,19
14,1991,1,22
10,1991,3,15
18,1991,4,3
0,1991,5,28
6,1991,7,2
8,1991,9,1
12,1991,11,11
4,1991,12,5
2,1991,12,16


In [51]:
start = test.sort_values(["year", "month", "day"]).iloc[0]
end = test.sort_values(["year", "month", "day"]).iloc[-1]

In [52]:
start

year     1991
month       1
day        19
Name: 16, dtype: int64

In [53]:
end

year     1995
month      12
day        16
Name: 5, dtype: int64

In [57]:
pd.to_datetime(test.sort_values(["year", "month", "day"]).iloc[[0, -1]])

16   1991-01-19
5    1995-12-16
dtype: datetime64[ns]

In [71]:
assert long.station.nunique() == 1
long.station.unique()[0]

'ACW00011604'